# Problem 13

## Problem 13.1

### R-10.1

Give a concrete implementation of the pop method in the context of the MutableMapping class, relying only on the five primary abstract methods of that class.

getitem , setitem , delitem , len , and iter are already there

In [2]:
from collections.abc import MutableMapping


class MapBase(MutableMapping):
    class _Item:
        __slots__ = "_key", "_value"

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
            return self._key < other._key


class UnsortedTableMap(MapBase):
    def __init__(self):
        self._table = []

    def __getitem__(self, k):
        for item in self._table:
            if k == item._key:
                return item._value
        raise KeyError("Key Error {0}".format(k))

    def __setitem__(self, k, v):
        for item in self._table:
            if k == item._key:
                item._value = v
                return
        self._table.append(self._Item(k, v))

    def __delitem__(self, k):
        for j in range(len(self._table)):
            if k == self._table[j]._key:
                self._table.pop(j)
                return
        raise KeyError("Key Error {0}".format(k))

    def __len__(self):
        return len(self._table)

    def __iter__(self):
        for item in self._table:
            yield item._key

    def pop(self, key):
        item = self[key]
        del self[key]
        return item

In [3]:
tb = UnsortedTableMap()
tb["one"] = 1
tb["two"] = 2
tb["three"] = 3

print(len(tb))

tb.pop("two")
print(len(tb))
try:
    tb["two"]
except KeyError:
    print("Deleted successfully!")

3
2
Deleted successfully!


### R-10.3

Give a concrete implementation of the `items( )` method directly within the `UnsortedTableMap` class, ensuring that the entire iteration runs in $O(n)$ time.

In [4]:
class UnsortedTableMap2(MapBase):
    def __init__(self):
        self._table = []

    def __getitem__(self, k):
        for item in self._table:
            if k == item._key:
                return item._value
        raise KeyError("Key Error {0}".format(k))

    def __setitem__(self, k, v):
        for item in self._table:
            if k == item._key:
                item._value = v
                return
        self._table.append(self._Item(k, v))

    def __delitem__(self, k):
        for j in range(len(self._table)):
            if k == self._table[j]._key:
                self._table.pop(j)
                return
        raise KeyError("Key Error {0}".format(k))

    def __len__(self):
        return len(self._table)

    def __iter__(self):
        for item in self._table:
            yield item._key

    def items(self):
        items = []
        for item in self._table:
            items.append(item._value)
        return items

In [7]:
tb = UnsortedTableMap2()
tb["one"] = 1
tb["two"] = 2
tb["three"] = 3

print(tb.items())

[1, 2, 3]


## Problem 13.2

### R-10.9 

Draw the 11-entry hash table that results from using the hash function, $h(i) = (3i + 5) \,\mathrm{ mod }\, 11$, to hash the keys $12$, $44$, $13$, $88$, $23$, $94$, $11$, $39$, $20$, $16$, and $5$, assuming collisions are handled by chaining.

In [ ]:
def h(i):
    return (3 * i + 5) % 11


for i in [12, 44, 13, 88, 23, 94, 11, 39, 20, 16, 5]:
    print(f"i = {i} -> h(i) = {h(i)}")

i = 12 -> h(i) = 8
i = 44 -> h(i) = 5
i = 13 -> h(i) = 0
i = 88 -> h(i) = 5
i = 23 -> h(i) = 8
i = 94 -> h(i) = 1
i = 11 -> h(i) = 5
i = 39 -> h(i) = 1
i = 20 -> h(i) = 10
i = 16 -> h(i) = 9
i = 5 -> h(i) = 9


| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10| 11|
|---|---|---|---|---|---|---|---|---|---|---|---|
| 13 | 94, 39 |   |   | 44, 88, 11 |   |   |  | 12, 23 | 16, 5 | 20 | |

### R-10.10

What is the result of the previous exercise, assuming collisions are handled by linear probing?

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10| 11|
|---|---|---|---|---|---|---|---|---|---|---|---|
| 13 | 94 | 39  | 5  | 44 | 88  | 11  |  | 12 | 23 | 20 | 16 |

## Problem 13.3

Test the ChainHashMap class for hash tables of different types of key-value pairs and different sizes. See the Resources.zip file for an implementation of ChainHashMap and its superclasses.

In [15]:
from resources.map_table import MapBase, UnsortedTableMap
from random import randrange


class HashMapBase(MapBase):
    def __init__(self, cap=11, p=109345121):
        self._table = cap * [None]
        self._n = 0
        self._prime = p
        self._scale = 1 + randrange(p - 1)
        self._shift = randrange(p)

    def _hash_function(self, k):
        return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)

    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table) // 2:
            self._resize(2 * len(self._table) - 1)

    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1

    def _resize(self, c):
        old = list(self.items())
        self._table = c * [None]
        self._n = 0
        for k, v in old:
            self[k] = v


class ChainHashMap(HashMapBase):
    def _bucket_getitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError("Key Error {0}".format(k))
        return bucket[k]

    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize:
            self._n += 1

    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError("Key Error {0}".format(k))
        del bucket[k]

    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:
                yield from bucket

In [ ]:
# Small Test for ChainHashMap with different types of key-value pairs and different sizes


def test_chain_hash_map():
    chm = ChainHashMap()
    chm["apple"] = 1
    chm["banana"] = 2
    chm[3] = "three"
    chm[(1, 2)] = "tuple_key"

    print("Items in ChainHashMap:")
    for key in chm:
        print(f"{key}: {chm[key]}")

    print("Length of ChainHashMap:", len(chm))

    del chm["banana"]
    print("After deleting 'banana':", len(chm))

    try:
        print(chm["banana"])
    except KeyError:
        print("'banana' successfully deleted!")
    chm[3] = "updated_three"
    print("Updated value for key 3:", chm[3])
    chm[(1, 2)] = "updated_tuple_key"

    print("Updated value for key (1, 2):", chm[(1, 2)])

    print("Final items in ChainHashMap:")
    for key in chm:
        print(f"{key}: {chm[key]}")


test_chain_hash_map()

Items in ChainHashMap:
apple: 1
banana: 2
3: three
(1, 2): tuple_key
Length of ChainHashMap: 4
After deleting 'banana': 3
'banana' successfully deleted!
Updated value for key 3: updated_three
Updated value for key (1, 2): updated_tuple_key
Final items in ChainHashMap:
apple: 1
3: updated_three
(1, 2): updated_tuple_key


## Problem 13.4

Review the generic hash table implementation in Algorithm 5. Explain why attribute `_n` is updated in `delitem(self, k)` but not updated in `setitem(self, k, v)`.

In [24]:
class HashMapBase(MapBase):
    def __init__(self, cap=11, p=109345121):
        self._table = cap * [None]
        self._n = 0
        self._prime = p
        self._scale = 1 + randrange(p - 1)
        self._shift = randrange(p)

    def _hash_function(self, k):
        return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)

    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table) // 2:
            self._resize(2 * len(self._table) - 1)

    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1

    def _resize(self, c):
        old = list(self.items())
        self._table = c * [None]
        self._n = 0
        for k, v in old:
            self[k] = v

`_n` is updated in delitem, because there the item gets removed every time, while in setitem the entry can be overwritten which would not increase the number of items in the hashtable.